In [ ]:
import torch
from  torchinfo import summary
import logging
import statistics
import numpy as np
import random
from tqdm import tqdm
logging.basicConfig(level="DEBUG")
# params =  torch.load("/home/od/Desktop/repos/dfd-clip/logs/multi_evidence/mix/ff+rppg-loo(NT)/best_weights.pt")
params =  torch.load("/home/od/Desktop/repos/dfd-clip/logs/multi_evidence/mix/ff+rppg-loo(NT)-c23/best_weights.pt")


In [ ]:
from main import get_config
c = get_config("configs/test.yml")
c

In [ ]:
from src.models import Detector
from accelerate import Accelerator

accelerator =  Accelerator(mixed_precision='no')

In [ ]:
model = Detector(c.model,c.data.num_frames,accelerator).eval().to("cuda")
model.load_state_dict(params)

In [ ]:
from src.datasets import RPPG
i,config = [ (i,config) for i,config in enumerate(c.data.eval) if config.name.lower()=="rppg"][0]
dataset = RPPG(config,c.data.num_frames,c.data.clip_duration,model.transform,accelerator,"val",i)
print(i)

In [ ]:
import numpy as np
import random
from tqdm import tqdm
distance = []
for _i in tqdm(range(len(dataset))):
    clip,label,mask,_ = dataset[_i]
    hearrate = np.argmax(label).tolist()
    with torch.no_grad():
        result = model.predict(clip.unsqueeze(0).to("cuda"),mask.unsqueeze(0).to("cuda"),dataset.index).cpu().tolist()
        torch.cuda.empty_cache()
    distance.append(abs(hearrate-np.argmax(result)))
print(statistics.mean(distance))
print(statistics.stdev(distance))

In [ ]:
from src.datasets import FFPP
report = {}
for t in ["REAL","DF","FS","F2F","NT"]:
    i,config = [ (i,config) for i,config in enumerate(c.data.train) if config.name.lower()=="deepfake"][0]
    config["types"] = [t]
    config["compressions"] = ["raw"]
    dataset = FFPP(config,c.data.num_frames,c.data.clip_duration,model.transform,accelerator,"val",i)
    hrs   = []
    for _i in tqdm(range(len(dataset))):
        clip,label,mask,_ = dataset[_i]
        with torch.no_grad():
            result = model.predict(clip.unsqueeze(0).to("cuda"),mask.unsqueeze(0).to("cuda"),0).cpu().tolist()
            torch.cuda.empty_cache()
        hrs.append(np.argmax(result))
    report[t] = [ statistics.mean(hrs), statistics.stdev(hrs)]
report